In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy # Add Deepcopy for args

import seaborn as sns 
import matplotlib.pyplot as plt
import os, pickle, joblib, argparse

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from statsmodels.tsa.api import SimpleExpSmoothing
# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
 
 
print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)

1.6.0+cpu
Populating the interactive namespace from numpy and matplotlib


In [2]:
# ================================================= #

# read file
raw_x_df = pd.read_csv('./data/train_x_df.csv')
raw_y_df = pd.read_csv('./data/train_y_df.csv')

print('Read files Complete!')
# ================================================= #


Read files Complete!


In [18]:
# # y dataframe time value에 1380 씩 adding
# raw_y_df.time = raw_y_df.time.copy() + 1380
# # x,y df merge하고 sample_id와 time 순으로 sorting
# merged_df = pd.concat([raw_x_df, raw_y_df])
# merged_df = merged_df.sort_values(by = ['sample_id','time']).reset_index(drop=True)


In [19]:
raw_y_arr = df2d_to_array3d(raw_y_df)

DataFrame to array, Complete!


In [23]:
open_arr = raw_y_arr[:, :, 1]

In [118]:
outlier_list = []
range_value = []

for idx, temp_arr in enumerate(open_arr):
    
    temp_min = temp_arr.min()
    temp_max = temp_arr.max()
    temp_arr_range = temp_max - temp_min
    range_value.append(temp_arr_range)

    if temp_arr_range > 0.05:
        outlier_list.append(idx)
        print(f'{idx}번째 open series is outlier sample!')
        print(f'temp array range is {temp_arr_range:.3}\n')
        
    

4번째 open series is outlier sample!
temp array range is 0.0519

6번째 open series is outlier sample!
temp array range is 0.0524

32번째 open series is outlier sample!
temp array range is 0.0582

58번째 open series is outlier sample!
temp array range is 0.0868

71번째 open series is outlier sample!
temp array range is 0.0538

79번째 open series is outlier sample!
temp array range is 0.105

83번째 open series is outlier sample!
temp array range is 0.136

96번째 open series is outlier sample!
temp array range is 0.23

134번째 open series is outlier sample!
temp array range is 0.062

147번째 open series is outlier sample!
temp array range is 0.105

150번째 open series is outlier sample!
temp array range is 0.0531

169번째 open series is outlier sample!
temp array range is 0.181

174번째 open series is outlier sample!
temp array range is 0.0675

177번째 open series is outlier sample!
temp array range is 0.124

205번째 open series is outlier sample!
temp array range is 0.068

206번째 open series is outlier sample!
temp ar

In [92]:
range_value_arr = np.array(range_value)

In [98]:
range_series = pd.Series(range_value_arr)

In [102]:
range_series.describe()['75%']

0.024538278579711803

In [109]:
range_series[range_series < range_series.describe()['75%']].shape

(5745,)

In [17]:
# ================================================= #
def df2d_to_array3d(df_2d):
    
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    
    print('DataFrame to array, Complete!')
    
    return array_3d
# ================================================= #

def train_tset_spliter(arr):
    n = len(arr)
    num_features = arr.shape[2]
    
    train_arr = arr[0:int(n*0.8), :, :]
    val_arr = arr[int(n*0.8):, :, :]
    
    train_len = len(train_arr) 
    val_len = len(val_arr)
    
    print(
    f'''
    ======================================================
    Origin length is {n}, then total split length is {train_len, val_len}
    ======================================================
    train length is {train_arr.shape},
    val length is {val_arr.shape},
    num_features is ({num_features})
    '''
    )
    return train_arr, val_arr
# ================================================= #

def kbin_discretizer(input_array):

    kb = KBinsDiscretizer(n_bins=10, strategy='uniform', encode='ordinal')
    processed_data = np.zeros((input_array.shape[0], input_array.shape[1], 1))
    
    for i in range(input_array.shape[0]):
        # coin_index_export args : (input_array, coin_num)
        globals()['processing_array{}'.format(i)] = input_array[i,:,1]
        
        #globals()['outliery_array{}'.format(i)] = train_y_array[outlier[i],:,1]
        kb.fit(globals()['processing_array{}'.format(i)].reshape(input_array.shape[1],1))
        globals()['processed_fit{}'.format(i)] = kb.transform(globals()['processing_array{}'.format(i)].reshape(input_array.shape[1],1))
        
        #globals()['outliery_fit{}'.format(i)] = kb.transform(globals()['outliery_array{}'.format(i)].reshape(120,1))
        processed_data[i,:,:] = globals()['processed_fit{}'.format(i)]
        
    return processed_data

In [4]:
# df to array 
raw_x_arr = df2d_to_array3d(raw_x_df)
raw_y_arr = df2d_to_array3d(raw_y_df)

# dis_x_arr = kbin_discretizer(raw_x_arr)
# dis_y_arr = kbin_discretizer(raw_y_arr)

# raw_x_arr[:, :, 1] = dis_x_arr.squeeze()
# dis_y_arr[:, :, 1] = dis_y_arr.squeeze()

DataFrame to array, Complete!
DataFrame to array, Complete!


In [5]:

# train test slit
train_x_arr, test_x_arr = train_tset_spliter(raw_x_arr)
train_y_arr, test_y_arr = train_tset_spliter(raw_y_arr)



    Origin length is 7661, then total split length is (6128, 1533)
    train length is (6128, 1380, 10),
    val length is (1533, 1380, 10),
    num_features is (10)
    

    Origin length is 7661, then total split length is (6128, 1533)
    train length is (6128, 120, 10),
    val length is (1533, 120, 10),
    num_features is (10)
    


In [6]:
# ================================================= #

def coin_index_export(input_array, coin_num):
    ''' 함수 설명 : 코인별 인덱스 뽑기 '''
    
    index = []
    sample_id_len = input_array.shape[0]
    coin_num_col = 0 

    for sample_id in range(sample_id_len):
        if input_array[sample_id, 0, coin_num_col] == coin_num:
            # print(sample_id)
            index.append(sample_id)
    return index

# ================================================= #

def each_coin_normalization(train_x_arr):
    ''' 함수 설명 : 코인별 데이터 정규화 '''
    
    # 유니크 코인 번호
    unique_coin_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    #create empty scaled list
    scaled_train_x_arr = np.zeros((train_x_arr.shape[0], train_x_arr.shape[1], train_x_arr.shape[2]))
    
    for temp_coin_num in unique_coin_index:
        # 유니크 코인 번호 중 한 코인 번호씩 해당 코인에 맞는 인덱스 추출
        # ex) if temp_coin_num is 0, temp_coin_index = [3, 7, 8, 14...]
        temp_coin_index = coin_index_export(train_x_arr, temp_coin_num)
        
        # temp coin num array export
        temp_x_arr = train_x_arr[temp_coin_index]
        
        # initialization
        num_sample   = temp_x_arr.shape[0] # sample dim
        num_sequence = temp_x_arr.shape[1] # time-sequence dim
        num_feature  = temp_x_arr.shape[2] # feature dim

        # create emptpy scaler
        temp_scaler = MinMaxScaler()
        
        # 시계열을 선회하면서 피팅합니다
        print('Current normalizing coin number is {}'.format(temp_coin_num))
        for temp_sample, temp_index in enumerate(temp_coin_index):
            temp_scaler.partial_fit(temp_x_arr[temp_sample, :, 5:]) # open =1, high = 2, low=3, close=4, volume=5 ~...

        # 스케일링(변환)합니다.
        for temp_sample, temp_index in enumerate(temp_coin_index):
            scaled_train_x_arr[temp_index, :, 5:] = temp_scaler.transform(temp_x_arr[temp_sample, :, 5:]).reshape(1, num_sequence, 5)
            scaled_train_x_arr[temp_index, :, :5] = temp_x_arr[temp_sample, :, :5]
            
        # save scaler for test arr
        dir_name = './scaler'
        file_name = f'coin_{temp_coin_num}_scaler.pkl'
        save_path = os.path.join(dir_name, file_name)
        joblib.dump(temp_scaler, save_path)
        
    
    print("Each coin normalization, Complete!")
    return scaled_train_x_arr

# ================================================= #

In [75]:
# train_x_arr, test_x_arr = train_tset_spliter(raw_x_arr)
# train_y_arr, test_y_arr = train_tset_spliter(raw_y_arr)

# 원래 하던 All coin 방식
# scaled_train_x_arr = each_coin_normalization(train_x_arr)

# 딥러닝 홀로서기의 normaliaztion 방식

Current normalizing coin number is 0
Current normalizing coin number is 1
Current normalizing coin number is 2
Current normalizing coin number is 3
Current normalizing coin number is 4
Current normalizing coin number is 5
Current normalizing coin number is 6
Current normalizing coin number is 7
Current normalizing coin number is 8
Current normalizing coin number is 9
Each coin normalization, Complete!


In [7]:
# ================================================= #

def simple_exponetial_smoothing_fory(arr, alpha=0.3):
    
    y_series = list()

    for temp_arr in arr:
        target_series = temp_arr[:, 1].reshape(-1) # open col is 1 index

        smoother = SimpleExpSmoothing(target_series, initialization_method="heuristic").fit(smoothing_level=alpha,optimized=False)
        smoothing_series = smoother.fittedvalues

        y_series.append(smoothing_series)
            
    return np.array(y_series)

# ================================================= #

def simple_exponetial_smoothing_forX(arr, alpha=0.3):
    
    # initialization
    sample_size = int(arr.shape[0])
    time_size = int(arr.shape[1])
    feature_size = int(arr.shape[2])
    
    # create empty array
    smoothing_arr = np.zeros((sample_size, time_size, feature_size - 1))

    for idx, temp_arr in enumerate(arr):
        for col in range(1, feature_size): # open col is 1 index
            if col < 5:

                temp_series = temp_arr[:, col].reshape(-1) 
                smoother = SimpleExpSmoothing(temp_series, initialization_method="heuristic").fit(smoothing_level=0.3,optimized=False)
                temp_smoothing_series = smoother.fittedvalues
                smoothing_arr[idx, :, col-1] = temp_smoothing_series

            else:
                
                pass_series = temp_arr[:, col].reshape(-1)
                smoothing_arr[idx, :, col-1] = pass_series

    return smoothing_arr

# ================================================= #

In [ ]:
# # train set smoothing
# train_x_arr = simple_exponetial_smoothing_forX(train_x_arr)
# train_y_arr = simple_exponetial_smoothing_fory(train_y_arr)

# # test set smoothing 
# test_x_arr = simple_exponetial_smoothing_forX(test_x_arr)
# print('simple exponetial smoothing Complete!!!')

In [8]:
print(
    f'''
    train x shape is {train_x_arr.shape},
    train y shape is {train_y_arr.shape},
    '''
    
#     test x shape is {test_x_arr.shape},
#     test y shape is {test_y_arr.shape},

)


    train x shape is (6128, 1380, 10),
    train y shape is (6128, 120, 10),
    


In [9]:
# train & val set
train_X, val_X, train_y, val_y = train_test_split(train_x_arr[:, :, 1:], train_y_arr[:, :, 1], test_size=0.1, shuffle=False)

# test set
# test_X = test_x_arr[:, :, 1:] # open col
# test_y = test_y_arr

print(
f'''
======================================================
Origin length is {len(train_x_arr)}, then total split length is {len(train_X)} + {len(val_X)} = {len(train_X)+len(val_X)}
======================================================
train X length is {train_X.shape}, train y length is {train_y.shape},
val X length is {val_X.shape}, val y length is {val_y.shape},
'''
# test X length is {test_X.shape}, test y length is {test_y.shape}
)


Origin length is 6128, then total split length is 5515 + 613 = 6128
train X length is (5515, 1380, 9), train y length is (5515, 120),
val X length is (613, 1380, 9), val y length is (613, 120),



In [17]:
# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

seed = 777
np.random.seed(seed)
torch.manual_seed(seed)


# ====== Model Capacity options ===== #a
args.input_dim = 9
args.hidden_dim = 50
args.output_dim = 1
args.n_layers = 1
args.batch_size = 8
args.dropout = 0.2
args.use_bn = True

# ====== Dataset Generating options ====== #
args.x_frames = 255
args.y_frames = 120

# ====== Model training options ===== #
args.num_epoch = 50
args.learning_rate = 0.0001
args.L2_rate = 0.00001


device is cpu


In [66]:
class WindowGenerator():
    ''' Dataset Generate'''
    def __init__(self, X_arr, y_arr, x_frames):
    
        self.X_arr = X_arr
        self.y_arr = y_arr
        self.x_frames = x_frames
        
        
    def __repr__(self):
        return '\n'.join([
            f'Input indices: {self.X_arr.shape}',
            f'Label indices: {len(self.y_arr)}',
            f'Current column name(s): {self.x_frames}'
        ])

    def __len__(self):
        return len(self.y_arr)

    def __getitem__(self, idx):
        
        X = self.X_arr[idx, -self.x_frames:, :]
        X = np.log(X + 1) - np.log(X[-1, :] + 1)
        
        y = self.y_arr[idx, :]
        y = np.log(y + 1) - np.log(y[-1] + 1)
        
        return X, y

In [67]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.dropout = dropout
        self.use_bn = use_bn 
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        self.regressor = self.make_regressor()
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, 200))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(200, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, X):
        lstm_out, self.hidden = self.lstm(X)
        y_pred = self.regressor(lstm_out[-1].view(X.shape[1], -1))
        return y_pred
    


    
def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
   
    # data load
    trainloader = DataLoader(partition['train'],
                             batch_size = args.batch_size,
                             shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()
    model.zero_grad()
    
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):
    
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y.float().to(args.device)
        
#         print(X.shape, y_true.shape)
        
        # zero the gradient
        optimizer.zero_grad()

        model.hidden = model.init_hidden(X.shape[1])

        y_pred = model(X)
#         print(y_pred.shape)
        
        loss = loss_fn(y_true.view(-1), y_pred.view(-1))
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()
        
    train_loss = train_loss / len(trainloader)
    # train_loss = train_loss*10E5
    return model, train_loss


def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], 
                           batch_size = args.batch_size, 
                           shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):
            
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y.float().to(args.device)
            
            model.hidden = model.init_hidden(X.shape[1])
            # en-decoder outputs tensor 
            y_pred = model(X)
            # compute the loss 
            loss = loss_fn(y_true.view(-1), y_pred.view(-1))

            # get the batch loss
            val_loss += loss.item()
            
    val_loss = val_loss / len(valloader)
    # val_loss = val_loss * 10E5
    return val_loss



def experiment(partition, args):


    model = LSTM(args.input_dim, args.hidden_dim, args.y_frames, args.n_layers, args.dropout, args.use_bn)
    model.to(args.device)
    
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []

    for epoch in range(args.num_epoch):
        
        start_time = time.time()
        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss = validate(model, partition, loss_fn, args)
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print('Epoch {},Loss(train/val) {:.3f}/{:.3f}. Took {:.2f} sec'.format(epoch+1, train_loss * 10E3, val_loss * 10E3, end_time-start_time))
    
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses #epoch 수에 의존
    result['val_losses'] = val_losses 
     
    return vars(args), result, model

In [68]:
trainset = WindowGenerator(train_X, train_y, x_frames = args.x_frames)
valset = WindowGenerator(val_X, val_y, x_frames = args.x_frames)
# testset = WindowGenerator(test_X, test_y, x_frames = args.x_frames)

partition = {'train': trainset, 'val':valset}

In [69]:
print(args)
setting, result, model = experiment(partition, deepcopy(args))

Namespace(L2_rate=1e-05, batch_size=8, device='cpu', dropout=0.2, hidden_dim=50, input_dim=9, learning_rate=0.0001, n_layers=1, num_epoch=50, output_dim=1, use_bn=True, x_frames=255, y_frames=120)


KeyboardInterrupt: 